# CVE Data Creator for Graph Analytics
**Quick data setup for Lecture 20 GraphFrames demo**

This notebook downloads and prepares CVEs for all of 2024!

In [0]:
import os
import json
import zipfile
import urllib.request
from pyspark.sql.functions import *

CATALOG = "workspace"
SCHEMA = "default"
VOLUME_NAME = "cve_demo"
DELTA_BRONZE_2024_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/cve_demo/bronze/2024_Nov13"
DELTA_SILVER_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/cve_demo/silver/cve_2024"

print(f"📁 2024 data will go to: {DELTA_BRONZE_2024_PATH}")

🚀 Setting up CVE data for Graph Analytics demo...
📁 2024 data will go to: /Volumes/workspace/default/cve_demo/bronze/2024_Nov13


In [0]:

print("📥 Downloading CVE repository...")

zip_dest = "/tmp/cve_graph_demo/cvelistV5.zip"
extract_dir = "/tmp/cve_graph_demo/cvelistV5-main"

os.makedirs("/tmp/cve_graph_demo", exist_ok=True)

# Download the repository
zip_url = "https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip"
print(f"📥 Downloading from: {zip_url}")

with urllib.request.urlopen(zip_url) as response:
    data = response.read()
    
with open(zip_dest, "wb") as f:
    f.write(data)

print(f"Downloaded {len(data):,} bytes")

# Extract the ZIP
print("Extracting ZIP archive...")
with zipfile.ZipFile(zip_dest) as z:
    z.extractall("/tmp/cve_graph_demo/")

print("Extraction complete")

📥 Downloading CVE repository...
📥 Downloading from: https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip
✅ Downloaded 524,600,171 bytes
📦 Extracting ZIP archive...
✅ Extraction complete


In [0]:
# Process 2024 CVEs
print("🎯 Processing 2024 CVEs...")

def process_year_cves(year, max_files=100000):
    """Process CVEs for a specific year, limiting to max_files"""
    
    cve_year_dir = f"{extract_dir}/cves/{year}"
    json_files = []
    
    print(f"Scanning directory: {cve_year_dir}")
    
    if os.path.exists(cve_year_dir):
        file_count = 0
        for root, dirs, files in os.walk(cve_year_dir):
            for file in files:
                if file.endswith('.json') and f'CVE-{year}-' in file and file_count < max_files:
                    file_path = os.path.join(root, file)
                    
                    try:
                        # Read and validate JSON
                        with open(file_path, 'r', encoding='utf-8') as f:
                            content = f.read()
                            cve_data = json.loads(content)
                            json_files.append(cve_data)
                            
                        file_count += 1
                        
                        if file_count % 500 == 0:
                            print(f"   Processed {file_count} CVE-{year} files...")
                            
                    except Exception as e:
                        print(f"Skipped {file}: {e}")
                        continue
                        
        print(f"Collected {len(json_files)} CVEs from {year}")
        return json_files
    else:
        print(f"Directory not found: {cve_year_dir}")
        return []

# Process both years
cves_2024 = process_year_cves(2024, 100000)

print(f"📊 Total 2024 CVEs: {len(cves_2024)}")


🎯 Processing 2024 CVEs...
📂 Scanning directory: /tmp/cve_graph_demo/cvelistV5-main/cves/2024
   📊 Processed 500 CVE-2024 files...
   📊 Processed 1000 CVE-2024 files...
   📊 Processed 1500 CVE-2024 files...
   📊 Processed 2000 CVE-2024 files...
   📊 Processed 2500 CVE-2024 files...
   📊 Processed 3000 CVE-2024 files...
   📊 Processed 3500 CVE-2024 files...
   📊 Processed 4000 CVE-2024 files...
   📊 Processed 4500 CVE-2024 files...
   📊 Processed 5000 CVE-2024 files...
   📊 Processed 5500 CVE-2024 files...
   📊 Processed 6000 CVE-2024 files...
   📊 Processed 6500 CVE-2024 files...
   📊 Processed 7000 CVE-2024 files...
   📊 Processed 7500 CVE-2024 files...
   📊 Processed 8000 CVE-2024 files...
   📊 Processed 8500 CVE-2024 files...
   📊 Processed 9000 CVE-2024 files...
   📊 Processed 9500 CVE-2024 files...
   📊 Processed 10000 CVE-2024 files...
   📊 Processed 10500 CVE-2024 files...
   📊 Processed 11000 CVE-2024 files...
   📊 Processed 11500 CVE-2024 files...
   📊 Processed 12000 CVE-2024 

In [0]:
import json
import pandas as pd
from pyspark.sql.functions import current_timestamp, current_date, lit, monotonically_increasing_id

def save_cves_to_delta_serverless(cves_list, year, delta_path):
    """Save CVEs to Delta Lake bronze layer - SERVERLESS COMPATIBLE"""
    if not cves_list:
        print(f"No CVEs to save for {year}")
        return
    
    print(f"📊 Converting {len(cves_list)} CVEs to DataFrame...")
    pdf = pd.DataFrame(cves_list)
    
    # Convert 'containers' dictionary to JSON string for Arrow compatibility
    if "containers" in pdf.columns:
        pdf["containers"] = pdf["containers"].apply(
            lambda x: json.dumps(x) if x is not None else None
        )
    
    # Convert any other potentially problematic nested columns
    # Check for other nested structures that might cause issues
    for col in pdf.columns:
        if pdf[col].dtype == 'object':
            # Check if first non-null value is dict or list
            first_val = pdf[col].dropna().iloc[0] if not pdf[col].dropna().empty else None
            if isinstance(first_val, (dict, list)):
                print(f"Converting nested column '{col}' to JSON string")
                pdf[col] = pdf[col].apply(
                    lambda x: json.dumps(x) if x is not None else None
                )
    
    df_raw = spark.createDataFrame(pdf)
    
    df_bronze = (
        df_raw
        .withColumn("_ingestion_timestamp", current_timestamp())
        .withColumn("_ingestion_date", current_date())
        .withColumn("_year", lit(year))
        .withColumn("_record_id", monotonically_increasing_id())
    )
    
    record_count = df_bronze.count()
    print(f"{year} CVE records: {record_count:,}")
    
    (
        df_bronze.write
        .format("delta")
        .mode("overwrite")
        .option("mergeSchema", "true")
        .option("overwriteSchema", "true")
        .option("delta.columnMapping.mode", "name")
        .save(delta_path)
    )
    
    print(f"{year} Bronze layer created: {delta_path}")
    return df_bronze

df_2024 = save_cves_to_delta_serverless(cves_2024, 2024, DELTA_BRONZE_2024_PATH)

📊 Converting 38753 CVEs to DataFrame...
Converting nested column 'cveMetadata' to JSON string
📈 2024 CVE records: 38,753
✅ 2024 Bronze layer created: /Volumes/workspace/default/cve_demo/bronze/2024_Nov13


In [0]:
  # Display DataFrame statistics
  record_count = df_2024.count()
  column_count = len(df_2024.columns)
  print(f" Total records: {record_count:,}")
  print(f"Total columns: {column_count}")

  # Test loading the bronze data for verification
  try:
      # Load the Delta table we just created
      bronze_2024 = spark.read.format("delta").load(DELTA_BRONZE_2024_PATH)

      print(f" 2024 Bronze data loaded: {bronze_2024.count():,} records")

      # Check the actual schema first
      print("\n Actual schema structure:")
      bronze_2024.printSchema()

      # Show sample data - use columns that exist as strings
      print("\n Sample 2024 data (available fields):")

      # Since cveMetadata is a JSON string, we need to parse it or just show raw columns
      from pyspark.sql.functions import get_json_object

      bronze_2024.select(
          get_json_object(col("cveMetadata"), "$.cveId").alias("cve_id"),
          get_json_object(col("cveMetadata"), "$.datePublished").alias("date_published"),
          col("_year").alias("year"),
          col("_ingestion_timestamp").alias("ingested_at")
      ).show(5, truncate=False)

      # Show available top-level columns
      print(f"\nAvailable columns: {bronze_2024.columns}")

      print(f"\n SUCCESS! Bronze layer ready for Silver transformation!")
      print(f" Bronze path: {DELTA_BRONZE_2024_PATH}")
      print(f" Registered table: bronze_2024")
      print(f"\n Query with: SELECT * FROM bronze_2024 LIMIT 10")
      print(f" Note: cveMetadata is stored as JSON string - parse in Silver layer")

  except Exception as e:
      print(f" Error verifying Bronze data: {e}")
      print("Showing basic table info instead...")

      # Fallback - just show basic info
      try:
          bronze_2024 = spark.read.format("delta").load(DELTA_BRONZE_2024_PATH)
          print(f" Table exists with {bronze_2024.count():,} records")
          print(f" Columns: {len(bronze_2024.columns)}")
          print(" Use Silver layer to parse JSON fields properly")
      except Exception as e2:
          print(f" Complete failure: {e2}")

🔍 Verifying Bronze layer data...
📊 Total records: 38,753
📊 Total columns: 8
✅ 2024 Bronze data loaded: 38,753 records

📋 Actual schema structure:
root
 |-- dataType: string (nullable = true)
 |-- dataVersion: string (nullable = true)
 |-- cveMetadata: string (nullable = true)
 |-- containers: string (nullable = true)
 |-- _ingestion_timestamp: timestamp (nullable = true)
 |-- _ingestion_date: date (nullable = true)
 |-- _year: integer (nullable = true)
 |-- _record_id: long (nullable = true)


🔍 Sample 2024 data (available fields):
+--------------+------------------------+----+--------------------------+
|cve_id        |date_published          |year|ingested_at               |
+--------------+------------------------+----+--------------------------+
|CVE-2024-44155|2024-10-28T21:08:25.991Z|2024|2025-11-16 19:45:39.479374|
|CVE-2024-44115|2024-09-10T03:08:43.205Z|2024|2025-11-16 19:45:39.479374|
|CVE-2024-44930|2024-08-29T00:00:00.000Z|2024|2025-11-16 19:45:39.479374|
|CVE-2024-44400|20

In [0]:
import shutil
try:
    shutil.rmtree("/tmp/cve_graph_demo")
    print(" Temporary files cleaned up")
except:
    print(" Temporary files will be cleaned up automatically")

print("\n CVE Data Creation Complete!")
print(" Ready for Graph Analytics with GraphFrames!")

🧹 Cleaning up temporary files...
✅ Temporary files cleaned up

🎉 CVE Data Creation Complete!
🔗 Ready for Graph Analytics with GraphFrames!


#Silver Layer

In [0]:
from pyspark.sql.functions import from_json, col, expr, regexp_extract, size
from pyspark.sql.types import StructType, StringType, StructField, ArrayType

# Define schema for cveMetadata
cve_metadata_schema = StructType([
    StructField("cveId", StringType()),
    StructField("datePublished", StringType()),
    StructField("dateUpdated", StringType()),
    StructField("state", StringType())
])

# Define schema for containers (simplified, adjust as needed)
cna_schema = StructType([
    StructField("descriptions", ArrayType(
        StructType([
            StructField("lang", StringType()),
            StructField("value", StringType())
        ])
    )),
    StructField("metrics", ArrayType(StringType())),
    StructField("affected", ArrayType(StringType())),
    StructField("references", ArrayType(StringType())),
    StructField("problemTypes", ArrayType(StringType()))
])
containers_schema = StructType([
    StructField("cna", cna_schema)
])

df_bronze = (
    df_bronze_read
    .withColumn("cveMetadata", from_json(col("cveMetadata"), cve_metadata_schema))
    .withColumn("containers", from_json(col("containers"), containers_schema))
)

df_silver = (
    df_bronze
    .select(
        col("cveMetadata.cveId").alias("cve_id"),
        col("cveMetadata.datePublished").alias("date_published"),
        col("cveMetadata.dateUpdated").alias("date_updated"),
        col("cveMetadata.state").alias("cve_state"),
        expr("transform(containers.cna.descriptions, x -> x.value)[0]").alias("description"),
        col("containers.cna.metrics").alias("cvss_metrics"),
        col("containers.cna.affected").alias("affected_products"),
        col("containers.cna.references").alias("references"),
        col("containers.cna.problemTypes").alias("problem_types"),
        col("_ingestion_timestamp"),
        col("_year")
    )
    .filter(col("cve_id").isNotNull())
    .filter(col("cve_id").startswith("CVE-2024-"))
    .withColumn("cve_number", expr("try_cast(regexp_extract(cve_id, 'CVE-\\d+-(\\d+)', 1) as int)"))
    .withColumn("has_description", col("description").isNotNull())
    .withColumn("has_cvss", col("cvss_metrics").isNotNull())
    .withColumn("reference_count", size(col("references")))
    .withColumn("affected_product_count", size(col("affected_products")))
)

silver_count = df_silver.count()
print(f"📊 Silver layer records: {silver_count:,}")

print("\n🔍 SILVER LAYER SAMPLE:")
display(
    df_silver.select(
        "cve_id", "date_published", "has_description", 
        "has_cvss", "reference_count", "affected_product_count"
    )
)

📊 Silver layer records: 38,753

🔍 SILVER LAYER SAMPLE:


cve_id,date_published,has_description,has_cvss,reference_count,affected_product_count
CVE-2024-0277,2024-01-07T13:00:05.458Z,true,true,3,1
CVE-2024-0340,2024-01-09T17:36:11.578Z,true,true,7,10
CVE-2024-0206,2024-01-09T12:56:46.957Z,true,true,1,1
CVE-2024-0166,2024-02-12T18:27:42.833Z,true,true,1,1
CVE-2024-0080,2024-04-05T17:51:30.536Z,true,true,1,1
CVE-2024-0550,2024-02-28T04:52:21.831Z,true,true,2,1
CVE-2024-0698,2024-03-05T01:55:59.881Z,true,true,2,1
CVE-2024-0647,2024-01-17T18:31:04.100Z,true,true,3,1
CVE-2024-0809,2024-01-23T23:53:01.615Z,true,false,4,1
CVE-2024-0755,2024-01-23T13:48:19.684Z,true,false,6,3


In [0]:
# Debug the path
print(f" Silver path: {DELTA_SILVER_PATH}")

# Ensure the volume exists using SQL (like your working Bronze approach)
try:
    spark.sql(f"""
    CREATE VOLUME IF NOT EXISTS workspace.default.{VOLUME_NAME}
    """)
    print(f" Volume '{VOLUME_NAME}' verified/created successfully")
except Exception as e:
    print(f" Volume may already exist: {e}")

# Now write the Silver layer
(df_silver.write
.format("delta")
.mode("overwrite")
.option("mergeSchema", "true")
.save(DELTA_SILVER_PATH))

# Register Silver table using TEMPORARY VIEW
spark.sql(f"""CREATE OR REPLACE TEMPORARY VIEW cve_2024_silver
AS SELECT * FROM delta.`{DELTA_SILVER_PATH}`""")

print(f" SILVER LAYER CREATED: {DELTA_SILVER_PATH}")
print(" Registered as temporary view: cve_2024_silver")

# Data quality report
print("\n SILVER LAYER DATA QUALITY REPORT:")
spark.sql("""SELECT 
    COUNT(*) as total_cves,
    SUM(CASE WHEN has_description THEN 1 ELSE 0 END) as cves_with_description,
    SUM(CASE WHEN has_cvss THEN 1 ELSE 0 END) as cves_with_cvss,
    AVG(reference_count) as avg_references,
    AVG(affected_product_count) as avg_affected_products
FROM cve_2024_silver""").show()

💾 WRITING SILVER LAYER TO DELTA LAKE...
📁 Silver path: /Volumes/workspace/default/cve_demo/silver/cve_2024
✅ Volume 'cve_demo' verified/created successfully
✅ SILVER LAYER CREATED: /Volumes/workspace/default/cve_demo/silver/cve_2024
📋 Registered as temporary view: cve_2024_silver

📊 SILVER LAYER DATA QUALITY REPORT:
+----------+---------------------+--------------+------------------+---------------------+
|total_cves|cves_with_description|cves_with_cvss|    avg_references|avg_affected_products|
+----------+---------------------+--------------+------------------+---------------------+
|     38753|                38032|         25993|2.2190260832982753|    1.955590029448885|
+----------+---------------------+--------------+------------------+---------------------+



#EDA

In [0]:
# 1. Monthly vulnerability counts
display(
    spark.sql("""
        SELECT _year, MONTH(date_published) AS publish_month, COUNT(*) AS vulnerability_count
        FROM cve_2024_silver
        WHERE date_published IS NOT NULL
        GROUP BY _year, publish_month
        ORDER BY _year, publish_month
    """)
)

_year,publish_month,vulnerability_count
2024,1,3320
2024,2,2834
2024,3,3573
2024,4,3484
2024,5,3781
2024,6,2835
2024,7,2991
2024,8,2794
2024,9,2483
2024,10,3410


In [0]:
# 2. Publication latency analysis (reserved vs. published dates)
display(
    spark.sql("""
        SELECT 
            cve_id,
            date_published,
            date_updated,
            DATEDIFF(date_updated,date_published) AS publication_latency_days
        FROM cve_2024_silver
        WHERE date_published IS NOT NULL AND date_updated IS NOT NULL
        ORDER BY publication_latency_days DESC
        LIMIT 50
    """)
)

cve_id,date_published,date_updated,publication_latency_days
CVE-2024-0193,2024-01-02T18:05:13.332Z,2025-11-06T20:51:53.548Z,674
CVE-2024-21634,2024-01-03T22:46:03.585Z,2025-11-03T21:53:26.420Z,670
CVE-2024-0209,2024-01-03T07:31:20.633Z,2025-11-03T21:50:52.188Z,670
CVE-2024-0562,2024-01-15T19:01:36.947Z,2025-11-15T08:17:20.196Z,670
CVE-2024-0211,2024-01-03T07:31:30.639Z,2025-11-03T21:50:53.647Z,670
CVE-2024-0208,2024-01-03T07:31:15.641Z,2025-11-03T21:50:50.735Z,670
CVE-2024-0340,2024-01-09T17:36:11.578Z,2025-11-07T17:06:21.545Z,668
CVE-2024-0443,2024-01-11T23:30:52.291Z,2025-11-07T13:08:14.692Z,666
CVE-2024-22368,2024-01-09T00:00:00.000Z,2025-11-04T22:05:37.486Z,665
CVE-2024-21647,2024-01-08T13:45:27.510Z,2025-11-03T21:53:27.921Z,665


In [0]:
# 3. Seasonal patterns in vulnerability disclosure (monthly trend)
display(
    spark.sql("""
        SELECT 
            MONTH(date_published) AS publish_month,
            COUNT(*) AS monthly_vulnerabilities
        FROM cve_2024_silver
        WHERE date_published IS NOT NULL
        GROUP BY publish_month
        ORDER BY publish_month
    """)
)

publish_month,monthly_vulnerabilities
1,3320
2,2834
3,3573
4,3484
5,3781
6,2835
7,2991
8,2794
9,2483
10,3410


In [0]:
%sql
SELECT
    CASE
        WHEN cvss_metrics IS NULL THEN 'Unknown'
        WHEN try_cast(get_json_object(cvss_metrics[0], '$.cvssV3_1.baseScore') AS FLOAT) >= 9 THEN 'Critical'
        WHEN try_cast(get_json_object(cvss_metrics[0], '$.cvssV3_1.baseScore') AS FLOAT) >= 7 THEN 'High'
        WHEN try_cast(get_json_object(cvss_metrics[0], '$.cvssV3_1.baseScore') AS FLOAT) >= 4 THEN 'Medium'
        ELSE 'Low'
    END AS cvss_bucket,
    COUNT(*) AS count
FROM cve_2024_silver
GROUP BY cvss_bucket
ORDER BY count DESC

cvss_bucket,count
Unknown,12760
Medium,11506
High,6750
Low,6106
Critical,1631


In [0]:
display(
    spark.sql("""
        SELECT
            _year,
            MONTH(date_published) AS publish_month,
            CASE
                WHEN cvss_metrics IS NULL THEN 'Unknown'
                WHEN try_cast(get_json_object(cvss_metrics[0], '$.cvssV3_1.baseScore') AS FLOAT) >= 9 THEN 'Critical'
                WHEN try_cast(get_json_object(cvss_metrics[0], '$.cvssV3_1.baseScore') AS FLOAT) >= 7 THEN 'High'
                WHEN try_cast(get_json_object(cvss_metrics[0], '$.cvssV3_1.baseScore') AS FLOAT) >= 4 THEN 'Medium'
                ELSE 'Low'
            END AS cvss_bucket,
            COUNT(*) AS count
        FROM cve_2024_silver
        WHERE date_published IS NOT NULL
        GROUP BY _year, publish_month, cvss_bucket
        ORDER BY _year, publish_month, cvss_bucket
    """)
)

_year,publish_month,cvss_bucket,count
2024,1,Critical,164
2024,1,High,530
2024,1,Low,359
2024,1,Medium,1095
2024,1,Unknown,1172
2024,2,Critical,128
2024,2,High,537
2024,2,Low,235
2024,2,Medium,1024
2024,2,Unknown,910


In [0]:
# 3. Unknown/unscored vulnerability identification
display(
    spark.sql("""
        SELECT cve_id, date_published, affected_products
        FROM cve_2024_silver
        WHERE cvss_metrics IS NULL
        LIMIT 50
    """)
)

cve_id,date_published,affected_products
CVE-2024-44155,2024-10-28T21:08:25.991Z,"List({""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""15"",""versionType"":""custom""}]}, {""vendor"":""Apple"",""product"":""iOS and iPadOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""17.7"",""versionType"":""custom""}]}, {""vendor"":""Apple"",""product"":""iOS and iPadOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""18"",""versionType"":""custom""}]}, {""vendor"":""Apple"",""product"":""watchOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""11"",""versionType"":""custom""}]}, {""vendor"":""Apple"",""product"":""Safari"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""18"",""versionType"":""custom""}]})"
CVE-2024-44930,2024-08-29T00:00:00.000Z,"List({""vendor"":""n/a"",""product"":""n/a"",""versions"":[{""version"":""n/a"",""status"":""affected""}]})"
CVE-2024-44400,2024-09-04T00:00:00,"List({""vendor"":""n/a"",""product"":""n/a"",""versions"":[{""version"":""n/a"",""status"":""affected""}]})"
CVE-2024-44939,2024-08-26T11:20:44.129Z,"List({""product"":""Linux"",""vendor"":""Linux"",""defaultStatus"":""unaffected"",""repo"":""https://git.kernel.org/pub/scm/linux/kernel/git/stable/linux.git"",""programFiles"":[""fs/jfs/jfs_dtree.c""],""versions"":[{""version"":""1da177e4c3f41524e886b7f1b8a0c1fc7321cac2"",""lessThan"":""f98bf80b20f4a930589cda48a35f751a64fe0dc2"",""status"":""affected"",""versionType"":""git""},{""version"":""1da177e4c3f41524e886b7f1b8a0c1fc7321cac2"",""lessThan"":""53023ab11836ac56fd75f7a71ec1356e50920fa9"",""status"":""affected"",""versionType"":""git""},{""version"":""1da177e4c3f41524e886b7f1b8a0c1fc7321cac2"",""lessThan"":""6ea10dbb1e6c58384136e9adfd75f81951e423f6"",""status"":""affected"",""versionType"":""git""},{""version"":""1da177e4c3f41524e886b7f1b8a0c1fc7321cac2"",""lessThan"":""9c2ac38530d1a3ee558834dfa16c85a40fd0e702"",""status"":""affected"",""versionType"":""git""},{""version"":""1da177e4c3f41524e886b7f1b8a0c1fc7321cac2"",""lessThan"":""ce6dede912f064a855acf6f04a04cbb2c25b8c8c"",""status"":""affected"",""versionType"":""git""}]}, {""product"":""Linux"",""vendor"":""Linux"",""defaultStatus"":""affected"",""repo"":""https://git.kernel.org/pub/scm/linux/kernel/git/stable/linux.git"",""programFiles"":[""fs/jfs/jfs_dtree.c""],""versions"":[{""version"":""5.15.189"",""lessThanOrEqual"":""5.15.*"",""status"":""unaffected"",""versionType"":""semver""},{""version"":""6.1.107"",""lessThanOrEqual"":""6.1.*"",""status"":""unaffected"",""versionType"":""semver""},{""version"":""6.6.47"",""lessThanOrEqual"":""6.6.*"",""status"":""unaffected"",""versionType"":""semver""},{""version"":""6.10.6"",""lessThanOrEqual"":""6.10.*"",""status"":""unaffected"",""versionType"":""semver""},{""version"":""6.11"",""lessThanOrEqual"":""*"",""status"":""unaffected"",""versionType"":""original_commit_for_fix""}]})"
CVE-2024-44271,2025-08-29T00:28:50.351Z,"List({""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""15.2"",""versionType"":""custom""}]})"
CVE-2024-44717,2024-08-29T00:00:00.000Z,"List({""vendor"":""n/a"",""product"":""n/a"",""versions"":[{""version"":""n/a"",""status"":""affected""}]})"
CVE-2024-44818,2024-09-04T00:00:00,"List({""vendor"":""n/a"",""product"":""n/a"",""versions"":[{""version"":""n/a"",""status"":""affected""}]})"
CVE-2024-44245,2024-12-11T22:57:37.431Z,"List({""vendor"":""Apple"",""product"":""visionOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""2.2"",""versionType"":""custom""}]}, {""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""15.2"",""versionType"":""custom""}]}, {""vendor"":""Apple"",""product"":""iPadOS"",""ve

In [0]:
# Vendor Intelligence

# 1. Top 25 vendors by vulnerability count
display(
    spark.sql("""
        SELECT
            vendor,
            COUNT(*) AS vulnerability_count
        FROM (
            SELECT EXPLODE(affected_products) AS vendor
            FROM cve_2024_silver
            WHERE affected_products IS NOT NULL
        )
        GROUP BY vendor
        ORDER BY vulnerability_count DESC
        LIMIT 25
    """)
)

vendor,vulnerability_count
"{""vendor"":""n/a"",""product"":""n/a"",""versions"":[{""version"":""n/a"",""status"":""affected""}]}",5901
"{""defaultStatus"":""affected"",""product"":""Adobe Experience Manager"",""vendor"":""Adobe"",""versions"":[{""lessThanOrEqual"":""6.5.20"",""status"":""affected"",""version"":""0"",""versionType"":""semver""}]}",161
"{""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""14.7"",""versionType"":""custom""}]}",133
"{""vendor"":""Google"",""product"":""Android"",""versions"":[{""version"":""Android kernel"",""status"":""affected""}],""defaultStatus"":""unaffected""}",119
"{""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""13.7"",""versionType"":""custom""}]}",113
"{""defaultStatus"":""affected"",""product"":""Adobe Experience Manager"",""vendor"":""Adobe"",""versions"":[{""lessThanOrEqual"":""6.5.21"",""status"":""affected"",""version"":""0"",""versionType"":""semver""}]}",100
"{""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""15"",""versionType"":""custom""}]}",99
"{""vendor"":""Microsoft"",""product"":""Windows Server 2022, 23H2 Edition (Server Core installation)"",""platforms"":[""x64-based Systems""],""versions"":[{""version"":""10.0.25398.0"",""lessThan"":""10.0.25398.1009"",""versionType"":""custom"",""status"":""affected""}]}",83
"{""vendor"":""Microsoft"",""product"":""Windows Server 2022"",""platforms"":[""x64-based Systems""],""versions"":[{""version"":""10.0.20348.0"",""lessThan"":""10.0.20348.2582"",""versionType"":""custom"",""status"":""affected""}]}",83
"{""vendor"":""Microsoft"",""product"":""Windows Server 2022, 23H2 Edition (Server Core installation)"",""platforms"":[""x64-based Systems""],""versions"":[{""version"":""10.0.25398.0"",""lessThan"":""10.0.25398.1189"",""versionType"":""custom"",""status"":""affected""}]}",83


In [0]:
# 2. Market concentration analysis (top vendors share)
display(
    spark.sql("""
        SELECT
            vendor,
            COUNT(*) AS vulnerability_count,
            ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS market_share_pct
        FROM (
            SELECT EXPLODE(affected_products) AS vendor
            FROM cve_2024_silver
            WHERE affected_products IS NOT NULL
        )
        GROUP BY vendor
        ORDER BY vulnerability_count DESC
        LIMIT 25
    """)
)

vendor,vulnerability_count,market_share_pct
"{""vendor"":""n/a"",""product"":""n/a"",""versions"":[{""version"":""n/a"",""status"":""affected""}]}",5901,7.93
"{""defaultStatus"":""affected"",""product"":""Adobe Experience Manager"",""vendor"":""Adobe"",""versions"":[{""lessThanOrEqual"":""6.5.20"",""status"":""affected"",""version"":""0"",""versionType"":""semver""}]}",161,0.22
"{""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""14.7"",""versionType"":""custom""}]}",133,0.18
"{""vendor"":""Google"",""product"":""Android"",""versions"":[{""version"":""Android kernel"",""status"":""affected""}],""defaultStatus"":""unaffected""}",119,0.16
"{""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""13.7"",""versionType"":""custom""}]}",113,0.15
"{""defaultStatus"":""affected"",""product"":""Adobe Experience Manager"",""vendor"":""Adobe"",""versions"":[{""lessThanOrEqual"":""6.5.21"",""status"":""affected"",""version"":""0"",""versionType"":""semver""}]}",100,0.13
"{""vendor"":""Apple"",""product"":""macOS"",""versions"":[{""version"":""unspecified"",""status"":""affected"",""lessThan"":""15"",""versionType"":""custom""}]}",99,0.13
"{""vendor"":""Microsoft"",""product"":""Windows Server 2022, 23H2 Edition (Server Core installation)"",""platforms"":[""x64-based Systems""],""versions"":[{""version"":""10.0.25398.0"",""lessThan"":""10.0.25398.1189"",""versionType"":""custom"",""status"":""affected""}]}",83,0.11
"{""vendor"":""Microsoft"",""product"":""Windows Server 2022"",""platforms"":[""x64-based Systems""],""versions"":[{""version"":""10.0.20348.0"",""lessThan"":""10.0.20348.2582"",""versionType"":""custom"",""status"":""affected""}]}",83,0.11
"{""vendor"":""Microsoft"",""product"":""Windows Server 2022, 23H2 Edition (Server Core installation)"",""platforms"":[""x64-based Systems""],""versions"":[{""version"":""10.0.25398.0"",""lessThan"":""10.0.25398.1009"",""versionType"":""custom"",""status"":""affected""}]}",83,0.11


In [0]:
display(
    spark.sql("""
        SELECT
            vendor,
            AVG(
                try_cast(
                    get_json_object(cvss_metrics[0], '$.cvssV3_1.baseScore') AS FLOAT
                )
            ) AS avg_cvss,
            COUNT(*) AS vulnerability_count,
            SUM(CASE WHEN cvss_metrics IS NULL THEN 1 ELSE 0 END) AS unknown_score_count
        FROM (
            SELECT EXPLODE(affected_products) AS vendor, cvss_metrics
            FROM cve_2024_silver
            WHERE affected_products IS NOT NULL
        )
        GROUP BY vendor
        ORDER BY avg_cvss DESC
        LIMIT 25
    """)
)

vendor,avg_cvss,vulnerability_count,unknown_score_count
"{""collectionURL"":""https://wordpress.org/plugins"",""defaultStatus"":""unaffected"",""packageName"":""wp-dummy-content-generator"",""product"":""WP Dummy Content Generator"",""vendor"":""Deepak anand"",""versions"":[{""changes"":[{""at"":""3.3.0"",""status"":""unaffected""}],""lessThanOrEqual"":""3.2.1"",""status"":""affected"",""version"":""n/a"",""versionType"":""custom""}]}",10.0,1,0
"{""vendor"":""pgjdbc"",""product"":""pgjdbc"",""versions"":[{""version"":""< 42.7.2"",""status"":""affected""},{""version"":""< 42.6.1"",""status"":""affected""},{""version"":""< 42.5.5"",""status"":""affected""},{""version"":""< 42.4.4"",""status"":""affected""},{""version"":""< 42.3.9"",""status"":""affected""},{""version"":""< 42.2.28"",""status"":""affected""}]}",10.0,1,0
"{""collectionURL"":""https://wordpress.org/plugins"",""defaultStatus"":""unaffected"",""packageName"":""jiangqie-free-mini-program"",""product"":""JiangQie Free Mini Program"",""vendor"":""酱茄"",""versions"":[{""lessThanOrEqual"":""2.5.2"",""status"":""affected"",""version"":""n/a"",""versionType"":""custom""}]}",10.0,1,0
"{""vendor"":""Synology"",""product"":""Unified Controller (DSMUC)"",""versions"":[{""version"":""3.1"",""status"":""affected"",""lessThan"":""3.1.4-23079"",""versionType"":""semver""},{""version"":""0"",""status"":""unknown"",""lessThan"":""3.1"",""versionType"":""semver""}],""defaultStatus"":""affected""}",10.0,1,0
"{""collectionURL"":""https://wordpress.org/plugins"",""defaultStatus"":""unaffected"",""packageName"":""instawp-connect"",""product"":""InstaWP Connect"",""vendor"":""InstaWP Team"",""versions"":[{""changes"":[{""at"":""0.1.0.39"",""status"":""unaffected""}],""lessThanOrEqual"":""0.1.0.38"",""status"":""affected"",""version"":""n/a"",""versionType"":""custom""}]}",10.0,1,0
"{""collectionURL"":""https://wordpress.org/plugins"",""defaultStatus"":""unaffected"",""packageName"":""audio-record"",""product"":""Audio Record"",""vendor"":""Dang Ngoc Binh"",""versions"":[{""lessThanOrEqual"":""1.0"",""status"":""affected"",""version"":""n/a"",""versionType"":""custom""}]}",10.0,1,0
"{""defaultStatus"":""unaffected"",""platforms"":[""Linux""],""product"":""Messaging Gateway"",""vendor"":""Symantec"",""versions"":[{""lessThanOrEqual"":""10.5"",""status"":""affected"",""version"":""0"",""versionType"":""semver""}]}",10.0,1,0
"{""vendor"":""Siemens"",""product"":""Sinteso FS20 EN X200 Cloud Distribution MP8"",""versions"":[{""status"":""affected"",""version"":""0"",""lessThan"":""V4.0.5016"",""versionType"":""custom""}],""defaultStatus"":""unknown""}",10.0,1,0
"{""vendor"":""xwiki"",""product"":""xwiki-platform"",""versions"":[{""version"":"">= 9.7-rc-1, < 15.10.11"",""status"":""affected""},{""version"":"">= 16.0.0-rc-1, < 16.4.1"",""status"":""affected""},{""version"":"">= 16.5.0-rc-1, < 16.5.0"",""status"":""affected""}]}",10.0,1,0
"{""vendor"":""thimpress"",""product"":""LearnPress – WordPress LMS Plugin"",""versions"":[{""version"":""*"",""status"":""affected"",""lessThanOrEqual"":""4.2.7"",""versionType"":""semver""}],""defaultStatus"":""unaffected""}",10.0,2,0
